In [ ]:
# from llama_index.llms.openai import OpenAI
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex 

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings 
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

import nest_asyncio

from dotenv import load_dotenv
load_dotenv()
nest_asyncio.apply()

# llm = OpenAI(model="gpt-4o-mini")
llm = GoogleGenAI(model="gemini-2.0-flash")


In [6]:
from tavily import AsyncTavilyClient


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key="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")
    return str(await client.search(query))

In [7]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that can search the web for information.",
)

In [8]:
response = await agent.run(user_msg="What is the weather in San Francisco?")
print(str(response))

The current weather in San Francisco is as follows:

- **Temperature**: 12.8°C (55.0°F)
- **Condition**: Mist
- **Wind**: 6.9 mph (11.2 kph) from the southwest
- **Humidity**: 89%
- **Visibility**: 16 km (9 miles)

For more details, you can check the full weather report [here](https://www.weatherapi.com/).


In [10]:
from tavily import AsyncTavilyClient
from llama_index.core.workflow import Context


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key="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")
    return str(await client.search(query))


async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic. Your input should be notes with a title to save the notes under."""
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."


async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic. Your input should be a markdown formatted report."""
    current_state = await ctx.get("state")
    current_state["report_content"] = report_content
    await ctx.set("state", current_state)
    return "Report written."


async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback. Your input should be a review of the report."""
    current_state = await ctx.get("state")
    current_state["review"] = review
    await ctx.set("state", current_state)
    return "Report reviewed."

In [11]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic. "
        "You should have at least some notes on a topic before handing off control to the WriteAgent."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review the write report and provide feedback. "
        "Your review should either approve the current report or request changes for the WriteAgent to implement. "
        "If you have feedback that requires changes, you should hand off control to the WriteAgent to implement the changes after submitting the review."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

In [12]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg=(
        "Viết report về các trường kinh tế ở Hà Nội "
        "Mô tả ngắn gọn ngành học, học phí  "
        "và triển vọng tương lai"
    )
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # if isinstance(event, AgentStream):
    #     if event.delta:
    #         print(event.delta, end="", flush=True)
    # elif isinstance(event, AgentInput):
    #     print("📥 Input:", event.input)
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")


🤖 Agent: ResearchAgent



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1504a2e70>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x150486ff0>, 200677.267774416)])']
connector: <aiohttp.connector.TCPConnector object at 0x1504a3500>


🛠️  Planning to use tools: ['search_web']
🔨 Calling Tool: search_web
  With arguments: {'query': 'các trường kinh tế ở Hà Nội'}
🔧 Tool Result (search_web):
  Arguments: {'query': 'các trường kinh tế ở Hà Nội'}
  Output: {'query': 'các trường kinh tế ở Hà Nội', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Các trường đại học kinh tế ở Hà Nội tốt nhất | JES.EDU.VN', 'url': 'https://jes.edu.vn/top-cac-truong-dai-hoc-kinh-te-tot-nhat-o-ha-noi', 'content': 'Danh sách các trường đại học có ngành kinh tế ở Hà Nội tốt nhất 1. Trường Đại học Kinh tế Quốc dân Trường Đại học Kinh tế Quốc dân là trường đại học hàng đầu Việt Nam đào tạo các khối ngành quản lý và kinh tế ở miền Bắc Việt Nam.', 'score': 0.9008183, 'raw_content': None}, {'title': 'Top 15 Trường đại học đào tạo ngành kinh tế tốt nhất Hà Nội - toplist.vn', 'url': 'https://toplist.vn/top-list/truong-dai-hoc-dao-tao-nganh-kinh-te-tot-nhat-ha-noi-15052.htm', 'content': 'Trên Hà Nội hiện nay có rất nhiều 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1504a1100>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x1504873b0>, 200682.504482833)])']
connector: <aiohttp.connector.TCPConnector object at 0x1504a3770>


🛠️  Planning to use tools: ['record_notes']
🔨 Calling Tool: record_notes
  With arguments: {'notes_title': 'Các trường kinh tế ở Hà Nội', 'notes': 'Các trường đại học kinh tế ở Hà Nội bao gồm: Trường Đại học Kinh tế Quốc dân, các trường đại học đào tạo ngành kinh tế tốt nhất Hà Nội, TOP 15 trường đại học kinh tế tại Hà Nội'}
🔧 Tool Result (record_notes):
  Arguments: {'notes_title': 'Các trường kinh tế ở Hà Nội', 'notes': 'Các trường đại học kinh tế ở Hà Nội bao gồm: Trường Đại học Kinh tế Quốc dân, các trường đại học đào tạo ngành kinh tế tốt nhất Hà Nội, TOP 15 trường đại học kinh tế tại Hà Nội'}
  Output: Notes recorded.


...

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15070caa0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x150487b30>, 200706.345632125)])']
connector: <aiohttp.connector.TCPConnector object at 0x15070e0c0>


🛠️  Planning to use tools: ['write_report']
🔨 Calling Tool: write_report
  With arguments: {'report_content': '# Các trường kinh tế ở Hà Nội\n\n## Giới thiệu\n\nHà Nội là một trung tâm lớn về đào tạo kinh tế ở Việt Nam, với nhiều trường đại học uy tín cung cấp các chương trình đào tạo đa dạng trong lĩnh vực này. Bài viết này sẽ giới thiệu một số trường kinh tế nổi bật ở Hà Nội, mô tả ngắn gọn về các ngành học, học phí, triển vọng tương lai, và các thông tin liên quan khác.\n\n## Các trường đại học kinh tế hàng đầu ở Hà Nội\n\n### 1. Trường Đại học Kinh tế Quốc dân (NEU)\n\n*   **Giới thiệu:** Trường Đại học Kinh tế Quốc dân là một trong những trường đại học hàng đầu về kinh tế và quản lý ở Việt Nam. Trường có bề dày lịch sử và đội ngũ giảng viên giàu kinh nghiệm.\n*   **Ngành học:** NEU cung cấp 60 ngành và chương trình đào tạo khác nhau, bao gồm:\n    *   Kinh tế\n    *   Quản trị kinh doanh\n    *   Tài chính - Ngân hàng\n    *   Kế toán\n    *   Marketing\n    *   Kinh tế quốc tế\n 

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15070c7a0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x150487b30>, 200709.847944708)])']
connector: <aiohttp.connector.TCPConnector object at 0x15070cf80>


📤 Output: The report is much improved with the addition of more schools and details. However, there are still areas for refinement:

*   **Fill in missing information:** The report mentions that the tuition fees for UEB-VNU, AOF, and TCU need to be researched. Please find and include this information.
*   **Add specific examples of international collaborations:** Instead of just stating that NEU has international collaborations, provide a few examples of partner universities or programs. Do the other schools have such collaborations as well?
*   **Expand on student life for each school:** Provide more details about the clubs, organizations, and activities available at each university. What makes the student experience unique at each institution?
*   **Add data on admission scores:** Where possible, include the average admission scores or cut-off scores for each school and program. This will help prospective students gauge their chances of acceptance.
*   **Improve the "So sánh các trườ

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x15070ddf0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x150487b30>, 200721.077609041)])']
connector: <aiohttp.connector.TCPConnector object at 0x15070de20>


In [14]:

state = await handler.ctx.get("state")
print(state["report_content"])

# Các trường kinh tế ở Hà Nội

## Giới thiệu

Hà Nội là một trung tâm lớn về đào tạo kinh tế ở Việt Nam, với nhiều trường đại học uy tín cung cấp các chương trình đào tạo đa dạng trong lĩnh vực này. Bài viết này sẽ giới thiệu một số trường kinh tế nổi bật ở Hà Nội, mô tả ngắn gọn về các ngành học, học phí, triển vọng tương lai, và các thông tin liên quan khác.

## Các trường đại học kinh tế hàng đầu ở Hà Nội

### 1. Trường Đại học Kinh tế Quốc dân (NEU)

*   **Giới thiệu:** Trường Đại học Kinh tế Quốc dân là một trong những trường đại học hàng đầu về kinh tế và quản lý ở Việt Nam. Trường có bề dày lịch sử và đội ngũ giảng viên giàu kinh nghiệm.
*   **Ngành học:** NEU cung cấp 60 ngành và chương trình đào tạo khác nhau, bao gồm:
    *   Kinh tế
    *   Quản trị kinh doanh
    *   Tài chính - Ngân hàng
    *   Kế toán
    *   Marketing
    *   Kinh tế quốc tế
    *   Thương mại điện tử
    *   ... và nhiều ngành khác.
*   **Học phí:** Học phí dự kiến khoảng 44 triệu đồng/năm (có thể thay

In [15]:
state["report_content"]

'# Các trường kinh tế ở Hà Nội\n\n## Giới thiệu\n\nHà Nội là một trung tâm lớn về đào tạo kinh tế ở Việt Nam, với nhiều trường đại học uy tín cung cấp các chương trình đào tạo đa dạng trong lĩnh vực này. Bài viết này sẽ giới thiệu một số trường kinh tế nổi bật ở Hà Nội, mô tả ngắn gọn về các ngành học, học phí, triển vọng tương lai, và các thông tin liên quan khác.\n\n## Các trường đại học kinh tế hàng đầu ở Hà Nội\n\n### 1. Trường Đại học Kinh tế Quốc dân (NEU)\n\n*   **Giới thiệu:** Trường Đại học Kinh tế Quốc dân là một trong những trường đại học hàng đầu về kinh tế và quản lý ở Việt Nam. Trường có bề dày lịch sử và đội ngũ giảng viên giàu kinh nghiệm.\n*   **Ngành học:** NEU cung cấp 60 ngành và chương trình đào tạo khác nhau, bao gồm:\n    *   Kinh tế\n    *   Quản trị kinh doanh\n    *   Tài chính - Ngân hàng\n    *   Kế toán\n    *   Marketing\n    *   Kinh tế quốc tế\n    *   Thương mại điện tử\n    *   ... và nhiều ngành khác.\n*   **Học phí:** Học phí dự kiến khoảng 44 triệu 